In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils 

import cv2
import glob

from PIL import Image
#from torchvision import transforms, utils
import random

In [2]:
mnist_test_input = np.load("MNIST/testImages.npy")

mnist_train_input = np.load("MNIST/trainImages.npy")




train_input = torch.Tensor(mnist_train_input.reshape(mnist_train_input.shape[0], 1, 28, 28)/128.0 - 1.0)#transform_to_tensor( transform, mnist_train_input)
test_input  = torch.Tensor(mnist_test_input.reshape(mnist_test_input.shape[0], 1, 28, 28)/128.0 - 1.0)#transform_to_tensor( transform, mnist_test_input)


In [20]:
# motivated by https://github.com/pytorch/examples/blob/master/dcgan/main.py
class Generator(nn.Module): 
    # input_size -> 28 x 28
    def __init__(self, input_size, channels):
        super(Generator, self).__init__()
        
        
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            # 28 -> 24
            
            nn.ConvTranspose2d(     input_size, channels * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(channels * 8),
            nn.ReLU(True),
            
            # 24 -> 11
            # state size. (channels*8) x 4 x 4
            nn.ConvTranspose2d(channels * 8, channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 4),
            nn.ReLU(True),
            
            # 
            # state size. (channels*4) x 8 x 8
            nn.ConvTranspose2d(channels * 4, channels * 2, 4, 2, 2, bias=False),
            nn.BatchNorm2d(channels * 2),
            nn.ReLU(True),
            # state size. (channels*2) x 16 x 16
            nn.ConvTranspose2d(channels * 2, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. 1 x 32 x 32
        )
    
    def forward(self, x): 
        
        ret = self.main(x)
        
        
        return ret
    

class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is 1 x 32 x 32
            nn.Conv2d(1, channels * 2, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(channels * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*2) x 16 x 16
            nn.Conv2d(channels * 2, channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*4) x 8 x 8
            nn.Conv2d(channels * 4, channels * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*8) x 4 x 4
            nn.Conv2d(channels * 8, 1, 3, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):    
        
        ret = self.main(x)
        
        
        return ret

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")    

chanNum = 32
nz = 100
    
netG = Generator(nz, chanNum).to(device)
netD = Discriminator(chanNum).to(device)
batch_size = 500
epoch = 300

trlr = 0.0001

optimizerD = optim.Adam(netD.parameters(), lr=trlr)
optimizerG = optim.Adam(netG.parameters(), lr=trlr)

criterion = nn.BCELoss()#F.binary_cross_entropy_with_logits

counter = 0


fn = torch.randn(1, nz, 1, 1, device=device)
cpu = torch.device("cpu")


for lll in range(epoch):
    print("epoch :" + str(lll + 1) )
    
    a = train_input.to(cpu).numpy()
    np.random.shuffle(a)
    train_input = torch.Tensor(a).to(device)
    
    for i in range(0, train_input.shape[0], batch_size):
        
    
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        
        data = train_input[i:i + batch_size]
        
        netD.zero_grad()
        real_cpu = data.to(device)
        
        label = torch.full((batch_size,), 1, device=device)
        label = label.view(-1, 1, 1, 1)
        
        output = netD(real_cpu)
        
        #output = output.view(-1)
        
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(1)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i%(batch_size*10) == 0:
            print(errD.item(), errG.item())
            
    
    output_sample = netG(fn).to(cpu).detach().numpy().reshape(28, 28)     
    print(np.max(output_sample))
    print(np.min(output_sample))
    cv2.imwrite("output/" + str(counter) + ".jpg",(output_sample + 1) * 128)
    counter = counter + 1
    


epoch :1
1.4626295566558838 0.6603779196739197
0.7047942876815796 1.0494036674499512
0.5662986040115356 1.3619266748428345
0.39759159088134766 1.9795199632644653
0.31399786472320557 2.164700746536255
0.19002431631088257 2.7288432121276855
0.15352550148963928 3.1354403495788574
0.12583081424236298 3.4693453311920166
0.07114087790250778 3.8362441062927246
0.09359890967607498 3.751173257827759
0.07147417962551117 3.815765142440796
0.10281120985746384 4.033472061157227
0.99342
-0.9994483
epoch :2
0.12385903298854828 3.6524369716644287
0.13381361961364746 3.676041603088379
0.10423324257135391 3.9638590812683105
0.10657995939254761 3.975480556488037
0.10137166827917099 3.9079749584198
0.10036813467741013 4.260615348815918
0.07720890641212463 4.203460216522217
0.1039714589715004 4.235053062438965
0.09111147373914719 4.465151786804199
0.043770886957645416 4.538542747497559
0.06787827610969543 4.271143436431885
0.10114654898643494 4.237943649291992
0.9584139
-0.99941415
epoch :3
0.0546396747231

0.048248540610075 5.776558876037598
0.04033402353525162 5.535193920135498
0.03479986637830734 5.8068318367004395
0.028077352792024612 5.506224155426025
0.999877
-1.0
epoch :18
0.050482697784900665 5.308375358581543
0.04725128039717674 5.168579578399658
0.05198337137699127 5.3470940589904785
0.09440281242132187 4.971074104309082
0.07934989780187607 5.605418682098389
0.07955574989318848 5.345146656036377
0.056117549538612366 4.984455108642578
0.06763707846403122 4.859174728393555
0.04488533362746239 5.1632161140441895
0.023682113736867905 5.836202144622803
0.038132812827825546 5.385379791259766
0.058644555509090424 4.772774696350098
0.9998116
-0.99999994
epoch :19
0.10904857516288757 5.083323955535889
0.06938585638999939 5.419792652130127
0.097187839448452 4.082843780517578
0.07108166813850403 4.716326713562012
0.0508500412106514 6.266869068145752
0.034158747643232346 5.03982400894165
0.047822367399930954 5.872321605682373
0.07467702776193619 4.746267318725586
0.04445236176252365 5.12217

0.23588350415229797 3.4493672847747803
0.27098578214645386 4.190647602081299
0.19040894508361816 2.9981794357299805
0.3491230309009552 3.3327889442443848
0.9994268
-1.0
epoch :35
0.15141229331493378 5.143860816955566
0.24973802268505096 4.072507381439209
0.33134421706199646 3.806403398513794
0.18569247424602509 4.875728607177734
0.18264000117778778 6.086228370666504
0.17203399538993835 5.083922386169434
0.3336351215839386 3.269204616546631
0.24891042709350586 3.330566644668579
0.340057373046875 3.093454360961914
0.2949763536453247 3.18479061126709
0.13522541522979736 3.941140651702881
0.10831136256456375 4.368397235870361
0.9986683
-1.0
epoch :36
0.10712215304374695 4.234483242034912
0.22935308516025543 3.0915567874908447
0.1990644335746765 4.1310038566589355
0.19186469912528992 4.500781536102295
0.1884651929140091 4.105637550354004
0.1284477561712265 4.562892913818359
0.17765261232852936 4.255821228027344
0.3081158399581909 3.6912899017333984
0.35128989815711975 3.010195016860962
0.16

0.1743221879005432 3.6557412147521973
0.3898015022277832 2.0535624027252197
0.5035929679870605 1.9919521808624268
0.9969831
-1.0
epoch :52
0.3742729127407074 2.527578830718994
0.2698696553707123 2.792750597000122
0.3406263589859009 2.5194098949432373
0.3339747190475464 2.2913286685943604
0.3478293716907501 2.2756710052490234
0.48075026273727417 2.5072693824768066
0.47199922800064087 2.272002935409546
0.24842548370361328 3.1979379653930664
0.44119513034820557 3.5920143127441406
0.6837342977523804 2.2022078037261963
0.291387140750885 2.9545936584472656
0.29424774646759033 3.0182158946990967
0.99840367
-1.0
epoch :53
0.45782890915870667 2.6565029621124268
0.40782004594802856 2.507920503616333
0.4694269597530365 2.262237548828125
0.41325467824935913 1.6035374402999878
0.48564133048057556 2.2710719108581543
0.3711363673210144 2.592223644256592
0.41752880811691284 2.690305709838867
0.39322593808174133 2.379807949066162
0.41300439834594727 3.348649263381958
0.251281201839447 3.53531813621521


0.6176913976669312 2.504950761795044
0.5361829996109009 2.0636215209960938
0.99837005
-1.0
epoch :69
0.3549783229827881 3.119910717010498
0.6761645078659058 1.3499243259429932
0.7169345617294312 1.6595386266708374
0.39769449830055237 2.000603437423706
0.4076746702194214 2.095740795135498
0.4287347197532654 1.9334397315979004
0.5716722011566162 2.0218887329101562
0.6586991548538208 2.0990161895751953
0.5034674406051636 2.4547276496887207
0.39928656816482544 2.294522762298584
0.739291787147522 2.556124448776245
0.5832052230834961 2.140650987625122
0.9989629
-1.0
epoch :70
0.32363635301589966 2.629333019256592
0.3951743245124817 2.0015692710876465
0.5882660150527954 2.0159530639648438
0.4751549959182739 2.3038806915283203
0.4041041433811188 2.592843532562256
0.4543842077255249 3.3391737937927246
0.7025131583213806 2.630185604095459
0.480724960565567 2.308751106262207
0.35194069147109985 2.4047722816467285
0.489706426858902 2.0341691970825195
0.6537657976150513 2.084393262863159
0.57727193

...

0.9972972
-1.0
epoch :223
0.6378797888755798 1.7556217908859253
0.5796796083450317 2.5666468143463135
0.6140990257263184 2.4597816467285156
0.5447801351547241 2.480128526687622
0.650862455368042 1.9014869928359985
0.5762830376625061 2.047517776489258
0.6519725322723389 1.9803026914596558
0.4633980393409729 1.8227999210357666
0.3662959933280945 2.5365800857543945
0.46408021450042725 2.4575133323669434
0.7490397691726685 1.8984507322311401
0.4999261200428009 2.815124988555908
0.9983153
-1.0
epoch :224
0.6152410507202148 2.126821756362915
0.502185046672821 2.7692337036132812
0.41961437463760376 2.468149185180664
0.5331363081932068 2.6132428646087646
1.0941240787506104 0.9620025157928467
0.3446105122566223 2.4834771156311035
0.37518733739852905 2.605968952178955
0.7052698135375977 1.9426367282867432
1.0204633474349976 1.9330707788467407
0.41291964054107666 2.742257833480835
0.29047006368637085 2.663910150527954
0.34683695435523987 2.4648220539093018
0.9978776
-1.0
epoch :225
0.710105061531

0.35800105333328247 2.3314871788024902
0.6068828105926514 1.947009801864624
0.7148341536521912 2.1861274242401123
0.7430381178855896 1.6076692342758179
0.44734853506088257 2.1252198219299316
0.42733240127563477 2.097169876098633
0.4708996117115021 1.5108678340911865
0.6829488277435303 1.4470174312591553
0.8736319541931152 1.4934544563293457
0.40778449177742004 1.999597430229187
0.43132004141807556 2.062166213989258
0.9990497
-1.0
epoch :241
0.7173497676849365 1.4845452308654785
0.6939777135848999 1.8817142248153687
0.608674168586731 2.2380740642547607
0.3999883532524109 2.664991855621338
0.3054097592830658 2.916860818862915
0.3437064290046692 2.64200758934021
0.7487130165100098 1.8253583908081055
0.7899001240730286 1.7146809101104736
0.43851715326309204 2.437495231628418
0.3715933561325073 2.52958083152771
0.5627208948135376 2.5273325443267822
0.7058082818984985 1.4120796918869019
0.99922025
-1.0
epoch :242
0.7038154006004333 1.7140390872955322
0.5065579414367676 1.9712940454483032
0.3

0.266743004322052 3.5789551734924316
0.283772736787796 2.6657843589782715
0.3650673031806946 2.905149459838867
0.6857311725616455 1.927683711051941
0.6828371286392212 1.6861568689346313
0.7021120190620422 2.231480121612549
0.3791552186012268 2.859375476837158
0.5402169227600098 2.4644603729248047
0.4193778932094574 2.4757535457611084
0.6576377749443054 2.3446364402770996
0.99865854
-1.0
epoch :258
0.5431913137435913 1.9019423723220825
0.4711381196975708 2.105794668197632
0.4961387515068054 2.941101551055908
0.3416171669960022 2.787904977798462
0.4378550946712494 1.8478212356567383
0.681948184967041 1.1328204870224
0.9036377668380737 1.5185203552246094
0.3481482267379761 2.6225392818450928
0.29803597927093506 2.0877740383148193
0.440779447555542 1.6802091598510742
0.6118113994598389 1.9952930212020874
0.7748901844024658 1.4417353868484497
0.99931866
-1.0
epoch :259
0.6952681541442871 2.188693046569824
0.4606582224369049 2.3877925872802734
0.46356773376464844 2.105888605117798
0.43354749

0.4554481506347656 2.2192420959472656
0.5583208799362183 2.78969669342041
0.6412893533706665 1.9749597311019897
0.5334956049919128 2.2666420936584473
0.3715462386608124 2.636059522628784
0.4864955544471741 2.907724618911743
1.0383872985839844 2.473668098449707
0.6657402515411377 2.524465799331665
0.3986608684062958 1.8372042179107666
0.99911875
-1.0
epoch :275
0.2936081886291504 2.4298605918884277
0.32032668590545654 2.957667589187622
0.5795004367828369 2.207714796066284
0.7207726836204529 1.4976378679275513
0.6525779962539673 2.326627016067505
0.673686146736145 1.7710216045379639
0.41949462890625 2.5633256435394287
0.4938615560531616 2.2706732749938965
0.5404563546180725 2.566110134124756
0.3873721957206726 3.4104697704315186
0.49512624740600586 2.4837496280670166
0.7403796911239624 1.2956064939498901
0.9991507
-1.0
epoch :276
0.539569616317749 2.5010905265808105
0.3886418342590332 2.1636428833007812
0.3884223699569702 2.79966139793396
0.2895545959472656 2.905466318130493
0.4928671717

0.4756646752357483 2.233158588409424
0.47588953375816345 2.4200644493103027
0.36121585965156555 2.8894786834716797
0.4200575053691864 2.777237892150879
0.42278677225112915 1.215882658958435
0.6386062502861023 3.39369535446167
0.49942827224731445 3.0356030464172363
0.33672165870666504 2.58138370513916
0.99969304
-1.0
epoch :292
0.29649117588996887 2.3197057247161865
0.4792224168777466 2.2903497219085693
0.5612772107124329 2.4908266067504883
0.4727833867073059 2.9083173274993896
0.5458468794822693 2.811112880706787
0.3279050588607788 2.670921802520752
0.4748959243297577 2.4579081535339355
0.4835837483406067 2.3042232990264893
0.6686000823974609 1.7354462146759033
0.6443012952804565 1.9809387922286987
0.3417026996612549 2.4395742416381836
0.3817020654678345 2.7488551139831543
0.9995374
-1.0
epoch :293
0.3172232210636139 2.466481924057007
0.43788814544677734 2.272759437561035
0.5423954725265503 2.288790702819824
0.6459012627601624 1.5567203760147095
0.3134725093841553 2.7271220684051514
0.

In [ ]:
cpu = torch.device( "cpu")    

noise = torch.randn(1, nz, 1, 1, device=device)


fake = netG(noise)

fake = fake.view(32, 32)

fakenp = fake.to(cpu).detach().numpy()



cv2.imwrite( "output/example.jpg", (fakenp + 1.0) * 128 )

In [21]:
torch.save(netG.state_dict(), "netG_MNIST_100")

